# Methods and Results

In [1]:
import altair as alt
import pandas as pd
import numpy as np
import sklearn
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import  plot_confusion_matrix, classification_report
random_state=77